## CS 418 - Introduction to Data Science Project Report
## Project : An Analysis on Chicago Crime & Real Estate
### Can we find the link between Chicago crime and real estate and predict future prices based on current crime record in each community?
**Team Crime Pricers (with GitHub handle)**: Cesar Lazcano (@clazca2, @cesarLazcano), Dheeraj Kondaparthi (@dkondaparthi), Iniobong Akpan (@Innyhandz), Jurat Shayiding (@jshayiding, @julaiti), HoangMinh HuynhNguyen (@jemiar)  
**Link to GitHub**: https://github.com/edgeslab/cs418-project-crime-pricers/blob/master/Final_Report.ipynb

## 1. Introduction
In this project, we tried to find the link between crime data and real estate data in 77 communities in Chicago. If there is a link, we tried to understand what their correlation looks like.  
In our investigation, we used, cleaned and combined crime data, geomap data from the City of Chicago, and real estate data from Redfin data center.  
Besides trying to understand crime and real estate data correlation, we also tried to:
* Build Machine Learning model to classify communities into 2 classes: price likely to rise, and price likely to drop based on crime data in each community
* Build Machine Learning model to predict real estate price (per square foot) for each community based on crime record from the previous year
* Build an interactive web application for users to explore crime and real estate data in Chicago from 2012 to present

## 2. Datasets and EDA
### 2.1 Crime dataset
* Source: Google Cloud BigQuery
* Structure: Tabular, csv formated
* Granularity: Each record represents 1 recorded crime
* Scope: Crimes recorded in Chicago, satisfy the scope of our project
* Temporality: From 2010 to 2018
* Faithfulness: Crimes were recorded by Chicago police officers. There are missing values, typos, different values for same block, crime types, .etc
#### Data cleaning: 
Due to the large crime data file, we use Google Cloud BigQuery to retrieve the summarized data we need for our project, and saved it to crime_data.csv (https://github.com/edgeslab/cs418-project-crime-pricers/blob/master/crime_data.csv)  
Data is grouped by community_area, year, and primary_type. Null community_area and primary_type are dropped. Community codes are guaranteed to be from 1. We're only interested in data from 2010 to 2018. The last column represents the number of crimes recorded of a crime type in 1 community in a year

In [ ]:
#Query to get summarized data using Google Cloud BigQuery
%%bigquery df

SELECT community_area, year, primary_type, COUNT(*) as count FROM `bigquery-public-data.chicago_crime.crime`
WHERE community_area IS NOT NULL AND year > 2009  AND year < 2019 AND community_area > 0 AND primary_type IS NOT NULL
GROUP BY community_area, year, primary_type
ORDER BY community_area, year, primary_type

We then pivot the data so that we have a dataframe that each row represents 1 community in a year. The columns are community_code, year, number of each crime type in a year.

In [1]:
import data_process
crimedf = data_process.process_crime()
crimedf.head(5)

,community,year,STALKING,CONCEALED CARRY LICENSE VIOLATION,CRIMINAL TRESPASS,LIQUOR LAW VIOLATION,BATTERY,OTHER NARCOTIC VIOLATION,HOMICIDE,THEFT,...,NON-CRIMINAL,ARSON,ROBBERY,PUBLIC INDECENCY,GAMBLING,KIDNAPPING,NON-CRIMINAL (SUBJECT SPECIFIED),WEAPONS VIOLATION,BURGLARY,PUBLIC PEACE VIOLATION
0,1,2011,4,0,162,14,994,0,4,1135,...,0,7,251,0,9,3,0,43,258,54
1,1,2012,3,0,183,21,991,0,6,1049,...,0,5,202,0,6,4,0,73,212,48
2,1,2013,2,0,190,11,866,0,4,1013,...,0,5,136,0,0,2,0,56,187,40
3,1,2014,2,0,141,8,794,0,8,828,...,1,1,117,0,0,2,0,40,181,22
4,1,2015,3,0,123,6,675,0,6,855,...,0,4,106,0,2,0,0,29,166,28


### 2.2 Real Estate dataset
* Source: Redfin data center https://www.redfin.com/blog/data-center
* Structure: Tabular, csv formated https://github.com/edgeslab/cs418-project-crime-pricers/blob/master/ppsf.csv
* Granularity: Each record represents 1 community or neighborhood
* Scope: Real estate data for 177 communities/neighborhoods in Chicago
* Temporality: From February 2012 to present
* Faithfulness: There are missing prices for some communities/neighborhoods. Names of communities and neigborhoods are mixed together
#### Data cleaning: 
We used the geojson file (https://github.com/edgeslab/cs418-project-crime-pricers/blob/master/chicago_communities.geojson) to find out which one are communities in the first column of our csv file.
For the other, we used Wikipedia (https://en.wikipedia.org/wiki/Community_areas_in_Chicago) to map neighborhoods to community. Our of total 77 communities in Chicago, we only have real estate data for 74 of them  
We then group our data by community names, and take the average price.
Finally we pivot our data to get the average price in a year.
Fortunately, when we did all the mapping and averaging, the missing values were not in the selection.
The final data frame represents average price per square foot for all 74 communities.

In [2]:
redf = data_process.process_realestate()
redf.head(5)

Community,year,albany park,archer heights,ashburn,auburn gresham,austin,avalon park,avondale,belmont cragin,beverly,...,washington heights,washington park,west elsdon,west englewood,west garfield park,west lawn,west pullman,west ridge,west town,woodlawn
0,2012,91.727273,91.954545,86.954545,32.454545,33.781818,36.936364,91.727273,61.909091,130.745455,...,37.490909,22.481818,97.418182,10.654545,12.154545,89.572727,20.127273,68.590909,201.536364,38.063636
1,2013,114.033333,111.183333,92.608333,30.900000,42.591667,37.758333,121.175000,59.400000,129.033333,...,42.987500,25.833333,98.775000,10.608333,11.058333,95.866667,21.133333,84.816667,229.566667,42.183333
2,2014,134.991667,112.866667,102.333333,34.066667,51.466667,46.733333,157.891667,94.666667,149.700000,...,52.733333,28.383333,115.275000,11.525000,17.550000,108.741667,22.325000,108.525000,255.750000,50.325000
3,2015,159.850000,117.991667,112.333333,40.391667,56.958333,53.833333,172.941667,89.683333,148.775000,...,57.991667,36.466667,118.050000,11.333333,16.583333,118.200000,25.250000,123.741667,268.848333,50.891667
4,2016,165.791667,119.541667,121.966667,49.375000,63.708333,65.225000,187.491667,120.033333,150.050000,...,65.633333,36.191667,129.783333,17.091667,26.625000,125.875000,31.208333,124.641667,272.541667,68.975000


## 3. Visualizations
 
### 3.1 A negative correlation: Crime drops, Price rises
As we can see from the graph below, in the majority of communities in Chicago, there is a negative correlation between the number of crimes recorded and the average price per square foot in those communities. There are only a handful of communities that have almost zero or positive correlation. Communities that has positive correlation mostly has a high price of real estate. We guess that police is more present in these areas, as well as residents there tend to report more crimes. (We didn't consider the seriousness of crime types)
![Correlation Graph](correlation_graph.png)

### 3.2 Good news for real estate investors in Chicago: Price is on the rise
As we can see from the graph below, Chicago real estate price is on the rise. The 1st quartile price, the median price and the 3rd quartile all increase through the year. However, we can't claim the reason is due to fall is crime alone because price rise can be attributed to multiple factors: inflation, the economy, the surrounding environment, .etc
![RealEstate Graph](re_dist.png)

## 4. Machine Learning models

## 5. Interactive Data Exploration
**Instruction to run the app**: https://github.com/edgeslab/cs418-project-crime-pricers/blob/master/README.md  
We developed an interactive data exploration app that allowed users to explore real estate and crime data by community. On the top left, we have a heatmap of price data, where the lighter the color, the more expensive price per square foot in that community. Oppositely, the darker the color, the cheaper the price. Users can use the year slider to see how the price heatmap changes over time. Users can also use the price range slider to filter out communities in a certain price range. The top right scatter plot depicts price and total number of crimes in a community, each dot represents a community. In addition, user can click on 1 community in the heatmap, and it will be highlighted in the scatter plot, as well as its crime trend and real estate trend will be shown on the 2 bottom time series graphs
![Heatmap and Scatter](app_images/crime&realEstate.PNG)  
Below is the crime exploration part. Here, crime is broken down into crime types. Users can select 1 or multiple crime types, and see their trend in the graph below.
![Crime breakdown](app_images/crime_line.PNG)

## 6. Results